In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [2]:
# 크롤링할 url 저장장
weapon = "https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass="
armor = "https://mhf.inven.co.kr/dataninfo/mhr/armor/"
accessory = "https://mhf.inven.co.kr/db/accessory/"
skill = "https://mhf.inven.co.kr/db/skill/"
url = [weapon, armor, accessory, skill]


In [3]:
# 무기 이름 가져와서 저장하기 
def create_wname(url):
    wname = []

    for i in range(4):
        print(url[i])
        resp = requests.get(url[i])
        if resp.status_code == 200:
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')

            #mhrDb > div.filter_form > div > table > tbody > tr:nth-child(1) > td > label.checkbox.active
            if i == 0:
                label = soup.select('#mhrDb > div.filter_form > div > table > tbody > tr:nth-child(1) > td > label.checkbox')
                for j in label:
                    wname.append(j.get_text())
        else:
            print(resp.status_code)
        return wname
# print(wname)
wname = create_wname(url)

https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=


In [4]:
print(wname)

['대검', '태도', '한손검', '쌍검', '해머', '수렵피리', '랜스', '건랜스', '슬래시액스', '차지액스', '조충곤', '라이트보우건', '헤비보우건', '활']


In [5]:
# 조합해서 추가적인 url 만들기 
weapon_list = []
for j in range(wname.__len__()):
    weapon_list.append(weapon + f'{j+1}')
weapon_list
url = [weapon_list, armor, accessory, skill]
print(url)

[['https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=1', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=2', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=3', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=4', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=5', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=6', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=7', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=8', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=9', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=10', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=11', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=12', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=13', 'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=14'], 'https://mhf.inven.co.kr/dataninfo/mhr/armor/', 'https://mhf.inven.co.kr/db/accessory/', 'https://mhf.inven.co.kr/db/skill/']


In [6]:
url[0].__len__()
url[0][13]  

'https://mhf.inven.co.kr/dataninfo/mhr/weapon/?subclass=14'

In [7]:
# html과 선택자를 받아서 원하는 부분을 뜯어오는 코드 
def list_create(soup, trsel):
    select_tr = soup.select_one(trsel)
    select_list = []
    
    td_data = select_tr.select('td')
    if td_data.__len__() == 0:
        td_data = select_tr.select('th')
        for i in range(td_data.__len__()):
            select_list.append(td_data[i].get_text())
    
    else:
        for i in range(td_data.__len__()):
            if td_data[i].get('class')[0] != 'sharpness':
                select_list.append(td_data[i].get('class')[0])
            
    
    return select_list

In [ ]:
df = pd.DataFrame()
df_list = []

armor_rare_list = []

for i in range(4):
    
    if i == 0:
        for j in range(url[i].__len__()):
            print(url[i][j])
            resp = requests.get(url[i][j])
            if resp.status_code == 200:
                # html 가져오기 
                html = resp.text
                soup = BeautifulSoup(html, 'html.parser')
                tr_len = soup.select('#mhrDb > div.mhr.board> table > tbody > tr')
                trsel = 'div.mhr.board > table > tbody > tr'
                herf = soup.select('#mhrDb > div.mhr.board> table > tbody > tr > td > a')
                
                select_index = list_create(soup, trsel)
                
                for k in range(select_index.__len__()):
                    globals()['{}'.format(select_index[k])] = []
                    
                rare_list = []
                for k in range(herf.__len__()):
                    url_plus = url[i][j][:45] + herf[k].get('href')
                    resp_plus = requests.get(url_plus)
                    html_plus = resp_plus.text
                    soup_plus = BeautifulSoup(html_plus, 'html.parser')
                    rare_score = soup_plus.select_one('div.mhr.weapon_filter.weapon_detail > table > tbody > tr:nth-child(2) > td:nth-child(4)').get_text()
                    
                    rare_list.append(rare_score)
                
                # 진행도 확인용
                num1 = 1
                con = tr_len.__len__() * num1 / 4
                for k in range(tr_len.__len__()):
                    for l in range(select_index.__len__()):
                        if select_index[l] != 'slot':
                            globals()['{}'.format(select_index[l])].append(soup.select_one(f'#mhrDb > div.mhr.board.board01 > table > tbody > tr:nth-child({k+1}) > td.{select_index[l]}').get_text().replace('\t', '').replace('\n', '').strip())
                        else:
                            globals()['{}'.format(select_index[l])].append(soup.select_one(f'#mhrDb > div.mhr.board.board01 > table > tbody > tr:nth-child({k+1}) > td.{select_index[l]}').get_text().replace('\t', '').replace('\n', '').strip()[3:])
                            
                    if con < k:
                        print('진행중...')
                        num1 += 1
                        con = tr_len.__len__() * num1 / 4
                    
                globals()['weapon_{}'.format(wname[j])] = pd.DataFrame()
                for k in range(select_index.__len__()):
                    globals()['weapon_{}'.format(wname[j])][f'{select_index[k]}'] = globals()['{}'.format(select_index[k])]
                
                globals()['weapon_{}'.format(wname[j])]['rare'] = rare_list
                df_list.append(f'weapon_{wname[j]}')
            else:
                print(resp.status_code)
    elif i == 1:
        print(url[i])
        resp = requests.get(url[i])
        if resp.status_code == 200:
            # html 가져오기 
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')
            tr_len = soup.select('#mhrDb > div.mhr.board> table > tbody > tr')
            trsel = 'div.mhr.board > table > tbody > tr'
            herf = soup.select('#mhrDb > div.mhr.board> table > tbody > tr > td > a')
            
            select_index = list_create(soup, trsel)
            
            for k in range(select_index.__len__()):
                globals()['{}'.format(select_index[k])] = []
            
            rare_list = []
            for k in range(herf.__len__()):
                url_plus = url[i][:45] + herf[k].get('href')
                resp_plus = requests.get(url_plus)
                html_plus = resp_plus.text
                soup_plus = BeautifulSoup(html_plus, 'html.parser')
                rare_score = soup_plus.select_one('div.mhr.armor_filter.armor_detail > table > tbody > tr:nth-child(2) > td:nth-child(2)').get_text()
                rare_list.append(rare_score)
            
            # 진행도 확인용
            num1 = 1
            con = tr_len.__len__() * num1 / 4
            
            for k in range(tr_len.__len__()):
                for l in range(select_index.__len__()):
                    globals()['{}'.format(select_index[l])].append(soup.select_one(f'#mhrDb > div.mhr.board.board01 > table > tbody > tr:nth-child({k+1}) > td.{select_index[l]}').get_text().strip().replace('\t', '').replace('\n', ','))
                    
                if con < k:
                    print('진행중...')
                    num1 += 1
                    con = tr_len.__len__() * num1 / 4
                
            globals()['armor_df'] = pd.DataFrame()
            for k in range(select_index.__len__()):
                globals()['armor_df'][f'{select_index[k]}'] = globals()['{}'.format(select_index[k])]
            globals()['armor_df']['rare'] = rare_list
            df_list.append('armor_df')
        else:
            print(resp.status_code)
    elif i == 2:
        print(url[i])
        resp = requests.get(url[i])
        if resp.status_code == 200:
            # html 가져오기 
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')
            tr_len = soup.select('table.list_table > tbody > tr')
            trsel = 'table.list_table > thead > tr'
            select_index = list_create(soup, trsel)
            
            for k in range(select_index.__len__()):
                globals()['{}'.format(select_index[k])] = []
            
            
            num1 = 1
            con = tr_len.__len__() * num1 / 4
            
            for k in range(tr_len.__len__()):
                for l in range(select_index.__len__()):
                    globals()['{}'.format(select_index[l])].append(soup.select_one(f'table.list_table > tbody > tr:nth-child({k+1}) > td:nth-child({l+1})').get_text().strip().replace('\t', '').replace('\n', ','))

                if con < k:
                    print('진행중...')
                    num1 += 1
                    con = tr_len.__len__() * num1 / 4
                
            globals()['accessory_df'] = pd.DataFrame()
            for k in range(select_index.__len__()):
                globals()['accessory_df'][f'{select_index[k]}'] = globals()['{}'.format(select_index[k])]
            df_list.append('accessory_df')
        else:
            print(resp.status_code)
    else:
        print(url[i])
        resp = requests.get(url[i])
        if resp.status_code == 200:
            # html 가져오기 
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')
            tr_len = soup.select('table.list_table > tbody > tr')
            trsel = 'table.list_table > thead > tr'
            select_index = list_create(soup, trsel)
            
            for k in range(select_index.__len__()):
                globals()['{}'.format(select_index[k])] = []
            
            # 진행도 확인용
            num1 = 1
            con = tr_len.__len__() * num1 / 4
            
            for k in range(tr_len.__len__()):
                for l in range(select_index.__len__()):
                    noness = soup.select_one(f'table.list_table > tbody > tr:nth-child({k+1}) > td:nth-child({l+1})').get_text().strip().replace('\t', '').replace('\n', ',').find('[')
                    if noness == -1:
                        globals()['{}'.format(select_index[l])].append(soup.select_one(f'table.list_table > tbody > tr:nth-child({k+1}) > td:nth-child({l+1})').get_text().strip().replace('\t', '').replace('\n', ','))
                    else:
                        globals()['{}'.format(select_index[l])].append(soup.select_one(f'table.list_table > tbody > tr:nth-child({k+1}) > td:nth-child({l+1})').get_text().strip().replace('\t', '').replace('\n', ',')[:noness])
                
                if con < k:
                    print('진행중...')
                    num1 += 1
                    con = tr_len.__len__() * num1 / 4
                
            globals()['skill_df'] = pd.DataFrame()
            for k in range(select_index.__len__()):
                globals()['skill_df'][f'{select_index[k]}'] = globals()['{}'.format(select_index[k])]
            df_list.append('skill_df')
        else:
            print(resp.status_code)
            print(i)
        
for i in df_list:
    globals()['update_{}'.format(i)].to_json(f'{i}')
print(df_list)

# 무기
#mhrDb > div.mhr.board.board01.board_melee > table > tbody > tr:nth-child(1)

# 방어구
#mhrDb > div.mhr.board.board01 > table > tbody > tr:nth-child(1)

# 장신구
#module_1732683949_2986_49563 > table > tbody > tr:nth-child(1)

# 무기
#module_1732683985_0141_10061 > table > tbody > tr:nth-child(1)


In [38]:
for i in df_list:
    globals()['{}'.format(i)].to_json(f'update_{i}.json', force_ascii=False, orient = 'records', indent=4)
print(df_list)

['weapon_대검', 'weapon_태도', 'weapon_한손검', 'weapon_쌍검', 'weapon_해머', 'weapon_수렵피리', 'weapon_랜스', 'weapon_건랜스', 'weapon_슬래시액스', 'weapon_차지액스', 'weapon_조충곤', 'weapon_라이트보우건', 'weapon_헤비보우건', 'weapon_활', 'armor_df']


In [10]:
skill_df

,이름,설명
0,KO술,"몬스터를 기절 상태로 만들 확률이 높아진다.,Lv.1 : 기절 위력 1.2배,Lv...."
1,가드 강화,"평소에는 가드할 수 없는 공격도 가드할 수 있게 된다.,Lv.1 : 대미지 경감 3..."
2,가드 배시 백룡,교체 기술 「가드」로 공격을 가드했을 때 크게 움츠리지 않은 경우 몬스터에게 타격 ...
3,가드 성능,"가드 시에는 좀처럼 움츠리지 않게 되고, 소비 스태미나도 줄어든다.,Lv.1 : 공..."
4,간파,"회심률이 상승한다.,Lv.1 : 회심률+5%,Lv.2 : 회심률+10%,Lv.3 :..."
...,...,...
176,회심격,"공격으로 회심이 발생했을 때 주는 속성 대미지(불, 물, 번개, 얼음, 용)가 높아..."
177,회피 거리 UP,"회피 시 이동 거리가 길어진다.,Lv.1 : 회피 거리가 조금 연장,Lv.2 : 회..."
178,회피 성능,"회피 시 무적 시간이 늘어난다.,Lv.1 : 무적 시간이 아주 조금 연장,Lv.2 ..."
179,흔들림 억제,"보우건으로 쏘는 탄환의 흔들림을 억제한다.,Lv.1 : 흔들림이 1단계 감소,Lv...."


In [11]:
accessory_df

,이름,슬롯Lv,레어도,스킬,생산 소재
0,절탄주II,2,10,탄환 절약 Lv2,"명왕원주 x 6,각명의 용혈옥 x 1,폭풍룡의 두툼비막 x 3"
1,혈기주II,3,10,혈기 Lv2,"명왕원주 x 6,라잔 하트 x 1,은작룡의 미려이빨 x 1"
2,격려주,1,10,격려 Lv1,"명왕원주 x 4,원초의 총애비늘 x 1,은작룡의 무구갑각 x 1"
3,달인주II,4,10,간파 Lv2,"괴이원주 x 6,빙룡의 영옥 x 1,파괴의 농용혈 x 6"
4,공격주II,4,10,공격 Lv2,"괴이원주 x 6,나락의 용신옥 x 1,파괴의 농용혈 x 6"
...,...,...,...,...,...
273,사법주,2,6,탄도 강화 Lv1,"수라원주 x 5,뇌신룡의 첨예발톱 x 2,뇌랑룡의 첨예발톱 x 3,각룡의 연수 x 1"
274,발도주,3,7,발도술[기] Lv1,"수라원주 x 4,풍신룡의 첨예발톱 x 3,양질의 비틀린뿔 x 2,호박색 이빨+ x 1"
275,이호주,3,7,버섯 애호가 Lv1,"수라원주 x 5,굉룡의 아기토 x 1,만악룡의 이빨+ x 3,만드라고라 x 3"
276,수세주,3,7,공격적인 방어 Lv1,"수라원주 x 5,풍신룡의 가죽+ x 3,적갑수의 첨예발톱 x 3,각룡의 연수 x 2"


In [12]:
armor_df

,name,type,defense,slot,elemental,skill
0,"프라그마헬름,여성용",머리,122,4 4 -,화:-2 수:2 뇌:-1 빙:2 용:-4,합기 Lv 2현여반석 Lv 2
1,프라이멀헬름,머리,125,4 4 -,화:-2 수:2 뇌:2 빙:-1 용:-5,혈기 Lv 3
2,"버추스헬름,남성용",머리,122,4 4 -,화:-2 수:2 뇌:-1 빙:2 용:-4,합기 Lv 2현여반석 Lv 2
3,황천[관],머리,125,4 3 -,화:-3 수:4 뇌:0 빙:0 용:-5,회피 성능 Lv 5
4,"네피림헬름,여성용",머리,118,4 2 -,화:-4 수:3 뇌:-2 빙:2 용:-4,분투 Lv 1간파 Lv 2약점 특효[속성] Lv 1
...,...,...,...,...,...,...
1306,이즈치그리브,다리,10,-,화:2 수:-1 뇌:-2 빙:0 용:0,회복 속도 Lv 1
1307,헌터그리브,다리,4,-,화:0 수:0 뇌:0 빙:0 용:0,공격 Lv 1기절 내성 Lv 1
1308,체인팬츠,다리,4,-,화:0 수:0 뇌:1 빙:0 용:0,스태미나 급속 회복 Lv 1
1309,레더팬츠,다리,1,-,화:0 수:0 뇌:0 빙:0 용:0,지질학 Lv 1


In [13]:
weapon_대검

,name,attack,critical,slot,option
0,파사의검 오무라사키,320,10,2 2 2백룡 1,불 28
1,칵투스크래프트,310,15,4 1 1백룡 3,불 65
2,오리진or엔드[2],340,-35/15,3 1 -백룡 2,용 51
3,빙익 세라피드[1],340,0,4 - -백룡 3,얼음 33
4,흉검[총운][2],360,-25,- - -백룡 3,물 58
...,...,...,...,...,...
275,스틸소드,100,0,- - -,
276,본블레이드 I,90,0,- - -,
277,본슬래셔,110,0,- - -,
278,수호단방어대검 I,110,0,- - -,방어+30
